<h1><center>  lab 9 : Text Classification </center>
    
<img src="https://miro.medium.com/max/1200/0*kHsYtU2d9IgyFwet.jpg" width="500">


```Created by Jinnie Shin (jinnie.shin@coe.ufl.edu)```\

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQmNf86oJnfhpkPA9LnrFnAbfwF2VywPYpB_w&usqp=CAU" align="left" width="70" height="70" align="left">

 ### Required Packages or Dependencies

In [2]:
#!pip install { } ! in case you run into the `package not avaialble` error
import pandas as pd
import numpy as np


## Dataset
#### Essay Set 10
---
`Prompt`: \
Brandi and Jerry were designing a doghouse. Brandi and Jerry did the following controlled experiment to find out how the color of an object affects its temperature. Use the results from the experiment to describe the best paint color for the doghouse.
In your description, be sure to:\
•	Choose a paint color. \
•	Describe how that color might affect the inside of the doghouse. \
•	Use results from the experiment to support your description.


> `1. Question`: What is the effect of different lid colors on the air temperature inside a glass jar exposed to a lamp? \
> `2. Hypothesis`: The darker the lid color, the greater the increase in air temperature in the glass jar, as darker colors absorb more energy. \
> `3. Materials`:
glass jar, lamp, four colored lids: {black, dark gray, light gray, and white}, thermometer, meterstick, stopwatch\
> `4. Procedures`:
>> 1.	Put the black lid with the attached thermometer on the glass jar.
>> 2.	Make sure the starting temperature inside the jar is 24° C.
>> 3.	Place lamp 5 centimeters away from the lid and turn on the lamp.
>> 4.	After 10 minutes measure the air temperature inside the glass jar and record as Trial 1.
>> 5.	Turn off lamp and wait until the air in the jar returns to the starting temperature.
>> 6.	Repeat steps 2 through 5 for Trials 2 and 3.
>> 7.	Repeat steps 1 through 6 for the dark gray, light gray, and white lids.
>> 8.	Calculate and record the average air temperature for each lid color.

> `5. Results`: Lid Colour vs. Air Temperature (Insides Glass Jar After 10 mins (°C)

| Lid Colour |   Trial 1|  Trial 1|   Trial 3| Average|
|------------|----------|----------|----------|----------|
| Black      | `54°C`   |`52°C`   |`54°C`   |`53°C`   |
| Dark gray  | `48°C`   | `48°C`   |`48°C`   | `48°C`   |
| Light gray | `44°C`   |`44°C`   |`46°C`   |`45°C`   |
| White      | `42°C`   | `43°C`   |`41°C`   | `42°C`   |

---
#### Essay Set 10 Data Description

| Type of response            | Source dependent response |
|-----------------------------|---------------------------|
| Grade level                 | `8`                       |
| Subject                     | `Science`                   |
| Total sample size           | `1,640`                     |
| Average length of responses | `60 words`                  |
| Score range                 | `0-2`                       |




In [3]:
df = pd.read_csv('train_rel_2.tsv', sep='\t')
df = df[df['EssaySet'] == 10]
df[:5]

,Id,EssaySet,Score1,Score2,EssayText
15403,25949,10,2,2,black :: black might affect the dog house beca...
15404,25950,10,1,1,white :: White because you wouldn't want the d...
15405,25951,10,2,2,white :: the color white will reflect the suns...
15406,25952,10,1,1,white :: this color will keep the dog house co...
15407,25953,10,2,2,white :: I would use a light color like White ...


<img src="https://i.pinimg.com/736x/2e/aa/7d/2eaa7d5021ca7c3c98bc93b98b9646fe.jpg" align="left" width="70" height="70" align="left">

 ## Task 1: Training data
>  Q1. Let's create a new column, called `color`, in the dataframe `df` that only saves the student's reponses regarding the colour choice (e.g., black, white, ... )  \
Hint: You can use the functions `.split()` and `.apply()`

> Q2: Remove the colour choice response from the `EssayText` column and save it into a new column `response`  \
(e.g., "black :: black might affect the dog ~ " -> "black might affect the dog~")

In [15]:
################################### YOUR CODE HERE for Q1 #############################
color = []

## Could use .apply(lamba x: x.split("::"))

for i in df['EssayText']:
  i_color = i.split("::")[0]
  color.append(i_color)

df['color'] = color
#######################################################################################
df['color'].value_counts()

white          733
black          392
light gray     288
dark gray      225
                 2
Name: color, dtype: int64

In [20]:
################################### YOUR CODE HERE for Q2 #############################
response = []

for i in df['EssayText']:
  i_resp = i.split("::")[1]
  response.append(i_resp)

df['response'] = response
#######################################################################################
print(df.response)

df = df.query('color != ""')

df['color'].value_counts()

15403     black might affect the dog house because then...
15404     White because you wouldn't want the dog to ge...
15405     the color white will reflect the suns rays of...
15406     this color will keep the dog house cooler ins...
15407     I would use a light color like White in this ...
                               ...                        
17038     white becuase if you live in a hot place the ...
17039     This color will affect the doghouse for the b...
17040     i think light gray would work the best becaus...
17041      if they painted the doghouse dark gray, then...
17042     the black would affect the doghouse because t...
Name: response, Length: 1638, dtype: object


white          733
black          392
light gray     288
dark gray      225
Name: color, dtype: int64

#### 1. Naive Bayes Classifier + BOW

In [33]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

### Let's create a BOW vector from the 'response' column as an input and call it `X`

from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X = bow.fit_transform(df['response']) ## Can use .values to convert to matrix, but sklearn can take pd

# ### Let's set the 'color' column as an input and call it `y`
y = df['color']

# ### Let's randomly shuffle and use 80% of the data as training and rest of it as testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(X_test.shape)

# #############################################################


clf_nb = MultinomialNB() # this is your new classifier
clf_nb .fit(X_train, y_train) #let's fit the model
y_hat = clf_nb.predict(X_test) #predit y_hat

# print(y_hat)

sklearn_score_test= clf_nb.score(X_test,y_test)
print("Sklearn's score on testing data :",sklearn_score_test)

print("Classification report for testing data : ")
print(classification_report(y_test, y_hat))


(1310, 2479)
(328, 2479)
Sklearn's score on testing data : 0.7347560975609756
Classification report for testing data : 
              precision    recall  f1-score   support

      black        0.80      0.58      0.67        69
  dark gray        0.50      0.30      0.37        37
 light gray        0.57      0.72      0.64        58
      white        0.81      0.90      0.86       164

    accuracy                           0.73       328
   macro avg       0.67      0.63      0.63       328
weighted avg       0.73      0.73      0.72       328



In [37]:
## Re-Do Above but for Score not Color Classification

bow = CountVectorizer()
X = bow.fit_transform(df['response']) ## Can use .values to convert to matrix, but sklearn can take pd

# print(df)

# ### Let's set the 'color' column as an input and call it `y`
y = df['Score1']

# ### Let's randomly shuffle and use 80% of the data as training and rest of it as testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(X_test.shape)

# #############################################################


clf_nb = MultinomialNB() # this is your new classifier
clf_nb .fit(X_train, y_train) #let's fit the model
y_hat = clf_nb.predict(X_test) #predit y_hat

# print(y_hat)

sklearn_score_test= clf_nb.score(X_test,y_test)
print("Sklearn's score on testing data :",sklearn_score_test)

print("Classification report for testing data : ")
print(classification_report(y_test, y_hat))

(1310, 2479)
(328, 2479)
Sklearn's score on testing data : 0.6798780487804879
Classification report for testing data : 
              precision    recall  f1-score   support

           0       0.73      0.12      0.20        68
           1       0.66      0.83      0.73       150
           2       0.71      0.83      0.76       110

    accuracy                           0.68       328
   macro avg       0.70      0.59      0.57       328
weighted avg       0.69      0.68      0.63       328



#### 2.Neural Network Classifier + BOW

In [ ]:
from sklearn.neural_network import MLPClassifier

clf_nn = MLPClassifier(random_state=1, max_iter=300) # this is your new classifier
clf_nn.fit(X_train,y_train) #let's fit the model
y_hat = clf_nn.predict(X_test) #predit y_hat

sklearn_score_test= clf_nn.score(X_test,y_test)
print("Sklearn's score on testing data :",sklearn_score_test)

print("Classification report for testing data : ")
print(classification_report(y_test, y_hat))

Sklearn's score on testing data : 0.7835365853658537
Classification report for testing data : 
              precision    recall  f1-score   support

      black        0.81      0.76      0.79        84
  dark gray        0.60      0.59      0.59        51
 light gray        0.70      0.64      0.67        58
      white        0.86      0.93      0.90       135

    accuracy                           0.78       328
   macro avg       0.74      0.73      0.74       328
weighted avg       0.78      0.78      0.78       328



#### 3.Support Vector Classifier + BOW

In [ ]:
from sklearn.svm import SVC

clf_svm = SVC(random_state=1, max_iter=3000) # this is your new classifier
clf_svm.fit(X_train,y_train) #let's fit the model
y_hat = clf_svm.predict(X_test) #predit y_hat

sklearn_score_test= clf_svm.score(X_test,y_test)
print("Sklearn's score on testing data :",sklearn_score_test)

print("Classification report for testing data : ")
print(classification_report(y_test, y_hat))

Sklearn's score on testing data : 0.7835365853658537
Classification report for testing data : 
              precision    recall  f1-score   support

      black        0.78      0.76      0.77        84
  dark gray        0.92      0.47      0.62        51
 light gray        0.71      0.78      0.74        58
      white        0.79      0.92      0.85       135

    accuracy                           0.78       328
   macro avg       0.80      0.73      0.75       328
weighted avg       0.79      0.78      0.78       328

